<a href="https://colab.research.google.com/github/rocifani/Simulacion-TPs/blob/main/Sist1Servidor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import simpy
import matplotlib.pyplot as plt

# Parámetros del sistema
tiempo_simulacion = 500000  # Tiempo total de simulación
tasa_llegada = 0.66 # Parámetro lambda de la distribución exponencial para la tasa de llegada
tasa_servicio = 0.7  # Parámetro mu de la distribución exponencial para la tasa de servicio

# Variables de rendimiento
num_clientes_atendidos = 0
num_denegaciones_servicio = 0
tiempo_total_sistema = 0
tiempo_total_cola = 0
tiempo_total_servicio = 0
num_clientes_en_cola = 0
tamano_cola = []

# Variables para el gráfico de evolución de la cola de espera
tiempos = []  # Lista para almacenar los momentos en los que cambia la cola
cola_espera = []  # Lista para almacenar el tamaño de la cola en cada momento

# Variables para el gráfico del estado del servidor
estados_servidor = []  # Lista para almacenar el estado del servidor (1: ocupado, 0: libre)

def cliente(env, nombre, servidor):
    llegada = env.now
    print(f'{nombre} llegó al sistema en el tiempo {llegada:.2f}')

    global num_clientes_en_cola, tiempos, cola_espera, tamano_cola, estados_servidor  # Declarar las variables globales

    with servidor.request() as req:
        yield req

        espera = env.now - llegada
        print(f'{nombre} comenzó a ser atendido en el tiempo {env.now:.2f} (espera: {espera:.2f})')

        servicio = random.expovariate(tasa_servicio)
        yield env.timeout(servicio)

        tiempo_salida = env.now
        print(f'{nombre} se fue en el tiempo {tiempo_salida:.2f} (servicio: {servicio:.2f})')

        global num_clientes_atendidos, tiempo_total_sistema, tiempo_total_cola, tiempo_total_servicio
        num_clientes_atendidos += 1
        tiempo_total_sistema += tiempo_salida - llegada
        tiempo_total_cola += espera
        tiempo_total_servicio += servicio
        num_clientes_en_cola -= 1

        tamano_cola.append(num_clientes_en_cola)  # Almacena el tamaño de la cola actual

        tiempos.append(env.now)  # Almacena el momento en que disminuye o aumenta la cola
        cola_espera.append(max(0, num_clientes_en_cola))  # Almacena el tamaño actual de la cola (mínimo 0)

        estados_servidor.append(0 if servidor.count == 0 else 1)  # Almacena el estado actual del servidor

# Configuración y ejecución de la simulación
env = simpy.Environment()
servidor = simpy.Resource(env, capacity=1)
cola_clientes = simpy.Resource(env, capacity=1)  # Cola de clientes

def llegada_clientes(env, servidor, cola_clientes):
    global num_clientes_en_cola, tiempos, cola_espera, tamano_cola, estados_servidor  # Declarar las variables globales
    i = 0
    while True:
        yield env.timeout(random.expovariate(tasa_llegada))
        i += 1
        env.process(cliente(env, f'Cliente {i}', servidor))
        num_clientes_en_cola += 1
        if i == 1:
            tiempos.append(env.now)  # Almacena el momento en que comienza el primer cliente
            cola_espera.append(0)  # La cola comienza en cero
        else:
            tiempos.append(env.now)  # Almacena el momento en que disminuye o aumenta la cola
            cola_espera.append(max(0, num_clientes_en_cola - 1))  # Almacena el tamaño actual de la cola (mínimo 0)
        estados_servidor.append(0 if servidor.count == 0 else 1)  # Almacena el estado actual del servidor

env.process(llegada_clientes(env, servidor, cola_clientes))
env.run(until=tiempo_simulacion)

# Resultados
tiempo_promedio_sistema = tiempo_total_sistema / num_clientes_atendidos
tiempo_promedio_cola = tiempo_total_cola / num_clientes_atendidos
tiempo_promedio_servicio = tiempo_total_servicio / num_clientes_atendidos
promedio_clientes_sistema = tiempo_total_sistema / tiempo_simulacion
promedio_clientes_cola = tiempo_total_cola / tiempo_simulacion
utilizacion_servidor = (tiempo_total_servicio / tiempo_simulacion) * 100

# Gráfico de la evolución de la cola de espera
plt.figure(figsize=(10, 6))
plt.step(tiempos, cola_espera, where='post')
plt.xlabel('Tiempo')
plt.ylabel('Clientes en cola de espera')
plt.title('Evolución de la cola de espera a lo largo del tiempo')
plt.grid(True)
plt.show()

# Histograma del estado del servidor
plt.figure(figsize=(10, 4))
plt.step(tiempos, estados_servidor, where='post')
plt.xlabel('Tiempo')
plt.ylabel('Estado del servidor')
plt.title('Estado del servidor a lo largo del tiempo')
plt.ylim(-0.1, 1.1)
plt.grid(True)
plt.show()

# Probabilidad de encontrar N clientes en cola (ingresado por teclado)
n = 2
probabilidad_n_clientes = cola_espera.count(n) / len(cola_espera)

# Resultados
probabilidad_denegacion_servicio = num_denegaciones_servicio / num_clientes_atendidos

print('\nResultados de la simulación:')
print(f'Número de clientes atendidos: {num_clientes_atendidos}')
print(f'Tiempo promedio en el sistema: {tiempo_promedio_sistema:.2f}')
print(f'Tiempo promedio en la cola: {tiempo_promedio_cola:.2f}')
print(f'Tiempo promedio de servicio: {tiempo_promedio_servicio:.2f}')
print(f'Promedio de clientes en el sistema: {promedio_clientes_sistema:.2f}')
print(f'Promedio de clientes en la cola: {promedio_clientes_cola:.2f}')
print(f'Porcentaje de utilización del servidor: {utilizacion_servidor:.2f}%')
print(f'Probabilidad de encontrar {n} clientes en cola: {probabilidad_n_clientes:.4f}')
print(f'Probabilidad de denegación de servicio: {probabilidad_denegacion_servicio:.4f}')

In [ ]:
!pip install simpy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import random
import math

# Parámetros del modelo
n = 120  # Número de meses
media_demanda = 0.45  # Media de los tiempos entre demandas en meses
probabilidades_demandas = [1/6, 1/3, 1/3, 1/6]  # Probabilidades de los tamaños de demanda
costo_base = 32  # Costo base de un pedido
costo_incremental = 3  # Costo incremental por cada artículo pedido
tiempo_entrega_min = 0.5  # Tiempo mínimo de entrega en meses
tiempo_entrega_max = 1  # Tiempo máximo de entrega en meses
costo_mantenimiento = 1  # Costo de mantenimiento por artículo por mes
costo_faltante = 5  # Costo de faltante por artículo por mes

# Variables de seguimiento
inventario = 0
costo_total = 0
articulos_en_falta = 0
momentos_demanda = []
momentos_pedido = []

for mes in range(1, n + 1):
    tiempo_demanda = random.expovariate(1 / media_demanda)
    tamano_demanda = random.choices([1, 2, 3, 4], probabilidades_demandas)[0]
    momentos_demanda.append((mes, tiempo_demanda, tamano_demanda))

    # Actualizar inventario según llegada de demanda
    if tamano_demanda <= inventario:
        inventario -= tamano_demanda
    else:
        articulos_en_falta += (tamano_demanda - inventario)
        inventario = 0

    # Verificar si se debe realizar un pedido
    if inventario < tamano_demanda:
        nivel_minimo_inventario = tamano_demanda
        costo_pedido = costo_base + costo_incremental * (nivel_minimo_inventario - inventario)
        tiempo_entrega = random.uniform(tiempo_entrega_min, tiempo_entrega_max)
        inventario += nivel_minimo_inventario
        costo_total += costo_pedido
        momentos_pedido.append((mes, tiempo_entrega))

    # Calcular costos de mantenimiento y faltante
    costo_mantenimiento = max(inventario, 0) * costo_mantenimiento
    costo_faltante = articulos_en_falta * costo_faltante

    costo_total += costo_mantenimiento + costo_faltante

    # Imprimir resultados del mes
    print(f"Mes {mes}:")
    print(f"Inventario: {inventario}")
    print(f"Costo de pedido: {costo_pedido}")
    print(f"Costo de mantenimiento: {costo_mantenimiento}")
    print(f"Costo de faltante: {costo_faltante}")
    print(f"Costo total: {costo_total}")
    print("------------------------------")

print(f"Costo total para los {n} meses: {costo_total}")
print("Momentos de demanda:")
for momento in momentos_demanda:
    print(f"Mes: {momento[0]}, Tiempo de demanda: {momento[1]}, Tamaño de demanda: {momento[2]}")
print("Momentos de pedido:")
for momento in momentos_pedido:
    print(f"Mes: {momento[0]}, Tiempo de entrega: {momento[1]}")



Mes 1:
Inventario: 3
Costo de pedido: 41
Costo de mantenimiento: 3
Costo de faltante: 15
Costo total: 59
------------------------------
Mes 2:
Inventario: 3
Costo de pedido: 35
Costo de mantenimiento: 9
Costo de faltante: 45
Costo total: 148
------------------------------
Mes 3:
Inventario: 3
Costo de pedido: 41
Costo de mantenimiento: 27
Costo de faltante: 135
Costo total: 351
------------------------------
Mes 4:
Inventario: 3
Costo de pedido: 41
Costo de mantenimiento: 81
Costo de faltante: 405
Costo total: 878
------------------------------
Mes 5:
Inventario: 3
Costo de pedido: 41
Costo de mantenimiento: 243
Costo de faltante: 1215
Costo total: 2377
------------------------------
Mes 6:
Inventario: 3
Costo de pedido: 35
Costo de mantenimiento: 729
Costo de faltante: 3645
Costo total: 6786
------------------------------
Mes 7:
Inventario: 3
Costo de pedido: 35
Costo de mantenimiento: 2187
Costo de faltante: 10935
Costo total: 19943
------------------------------
Mes 8:
Inventario: 3